In [1]:
import nltk
import numpy as np
import pandas as pd
from pandas import DataFrame
from keras.models import Model, Sequential, load_model
from keras.preprocessing.sequence import TimeseriesGenerator, pad_sequences
from keras.layers.recurrent import LSTM
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers.core import Activation, Dropout, Dense, Flatten
from keras.layers import TimeDistributed, Bidirectional, InputLayer, GlobalMaxPooling1D
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.callbacks import TensorBoard
from keras.metrics import categorical_accuracy
from IPython.display import clear_output
from more_itertools import flatten, intersperse
import random

import os
import urllib.request

Using TensorFlow backend.


In [2]:
%load_ext autoreload
%autoreload 2
from batcher import batch_from_generator
from train_data import load_conll2003, create_conll_encoded_shifted_generator
from mappings import get_all_mappings, gen_input_feature_to_class_map, gen_input_feature_to_int_map
from corpus import corpus_training_data_generator, create_all_corpus_train_pipeline, pad, encode_each_sentence
from model import create_model, compile_model

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\Todd\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package reuters to
[nltk_data]     C:\Users\Todd\AppData\Roaming\nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\Todd\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [3]:
DROPOUT = 0.5
TIME_SLICE_SIZE = 50
BATCH_SIZE = 512
SAMPLING_RATE = 1
OUTPUT_CLASSES = 2
PADDING = 0
UNKNOWN = 1
NUM_OF_UNITS = 200
EPOCHS=200
MODEL_SAVE_PATH = 'tc_model.h5'
LSTM_MODEL_SAVE_PATH = 'lstm_model.h5'

In [4]:
mapping, reverse_mapping, lower_mapping, lower_reverse_mapping = get_all_mappings()
input_feature_to_class_map = gen_input_feature_to_class_map()
input_feature_to_int_map = gen_input_feature_to_int_map()

In [5]:
# def create_model(shape, classes):
#     # First layer inputs must be 3D
#     # with shape (samples, timesteps, features)
#     model = Sequential()
#     model.add(InputLayer(input_shape=shape))
#     model.add(Bidirectional(LSTM(NUM_OF_UNITS, return_sequences=True, dropout=DROPOUT, recurrent_dropout=DROPOUT)))
#     model.add(TimeDistributed(Dense(1000)))
#     model.add(TimeDistributed(Dense(classes)))
#     model.add(Activation('softmax'))
#     return model

In [6]:
model = create_model(1, 2, NUM_OF_UNITS, DROPOUT)

In [7]:
model = compile_model(model)

In [8]:
if os.path.isfile(MODEL_SAVE_PATH): 
    model.load_weights(MODEL_SAVE_PATH)

In [9]:
tensor_board = TensorBoard(batch_size=BATCH_SIZE, write_graph=False, )

In [10]:
X, Y = create_all_corpus_train_pipeline(TIME_SLICE_SIZE)

In [11]:
W = Y * 10
W = W + 1
W = W.reshape((-1, 50))

In [12]:
W.shape

(115001, 50)

In [13]:

#g = corpus_training_data_generator('gutenberg',TIME_SLICE_SIZE, BATCH_SIZE, shift=False)
model.fit(X, Y, validation_split=0.2, callbacks=[tensor_board], batch_size=BATCH_SIZE, epochs=EPOCHS, sample_weight=W)

Train on 92000 samples, validate on 23001 samples
Epoch 1/200
92000/92000 [==============================] - 48s 526us/step - loss: 0.5617 - acc: 0.9193 - val_loss: 0.3531 - val_acc: 0.9179
Epoch 2/200
92000/92000 [==============================] - 45s 493us/step - loss: 0.3624 - acc: 0.9363 - val_loss: 1.3732 - val_acc: 0.7245
Epoch 3/200
92000/92000 [==============================] - 45s 494us/step - loss: 0.3099 - acc: 0.9471 - val_loss: 1.5959 - val_acc: 0.7607
Epoch 4/200
92000/92000 [==============================] - 45s 494us/step - loss: 0.2850 - acc: 0.9517 - val_loss: 1.9980 - val_acc: 0.7379
Epoch 5/200
92000/92000 [==============================] - 45s 494us/step - loss: 0.2737 - acc: 0.9543 - val_loss: 2.3716 - val_acc: 0.7147
Epoch 6/200
92000/92000 [==============================] - 45s 494us/step - loss: 0.2653 - acc: 0.9556 - val_loss: 2.3227 - val_acc: 0.7099
Epoch 7/200
92000/92000 [==============================] - 45s 494us/step - loss: 0.2610 - acc: 0.9564 - val_l

In [29]:
original_sentence = "Tim Smith works for Google in California".lower()
test_sentence = pad([list(original_sentence)], len(original_sentence))
test_sentence = encode_each_sentence(test_sentence, input_feature_to_int_map)

mapped_sentence = np.asarray(test_sentence)
predicted_result = model.predict_classes(mapped_sentence)[0]
print(predicted_result)
predicted_result = list(zip(pad([list(original_sentence)], len(original_sentence))[0], predicted_result.tolist()))

def true_case(letter, label):
    if letter == 0:
        return ''
    if(label == 0):
        return letter.lower()
    if (label == 1):
        return letter.upper()
    return letter

predicted_result = [true_case(letter, label) for letter, label in predicted_result]
''.join(predicted_result) 



[1 0 0 1 1 0 0 0 0 1 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 0]


'Tim Smith Works for google in california'

In [15]:
model.save("big_lstm.h5")

In [16]:

for x in X[0]:
    print(x )

[63.]
[56.]
[27.]
[36.]
[54.]
[20.]
[2.]
[50.]
[2.]
[44.]
[23.]
[42.]
[57.]
[61.]
[20.]
[33.]
[2.]
[50.]
[2.]
[20.]
[28.]
[28.]
[42.]
[2.]
[64.]
[30.]
[56.]
[45.]
[49.]
[30.]
[36.]
[34.]
[20.]
[3.]
[2.]
[23.]
[16.]
[29.]
[45.]
[60.]
[30.]
[28.]
[20.]
[3.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]


In [17]:
from sklearn.utils import class_weight

In [18]:
len(Y.shape)

3